In [1]:
!pip install numpy
!pip install torch
import numpy #1.24
print(numpy.__version__)

!pip install transformers[torch]
!pip install transformers
!pip install scikit-learn
!pip install sentencepiece
!pip install protobuf

!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu118



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


1.23.5



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu118



[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import sys
import torch

if torch.cuda.is_available():
    device = torch.device("cuda")
    print('GPU disponivel')
    print('Número de GPUs disponíveis:', torch.cuda.device_count())
    print('Nome da GPU:', torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print('GPU não disponível')

GPU disponivel
Número de GPUs disponíveis: 1
Nome da GPU: NVIDIA GeForce RTX 2070


In [2]:
import json
from sklearn.model_selection import train_test_split

with open('dataset.jsonl', 'r', encoding='utf-8') as f:
    data = [json.loads(line) for line in f]

inputs = [item['input'] for item in data]
outputs = [
    f"code: {item['output']['catalog_catalog.code']} description: {item['output']['catalog_catalog.description']} "
    f"full_description: {item['output']['catalog_catalog.full_description']} similar_parts: {item['output']['catalog_catalog.similar_parts']} "
    f"application_table: {item['output']['catalog_catalog.application_table']} brand_producer: {item['output']['catalog_catalog.brand_producer']}"
    for item in data
]


In [3]:
train_inputs, test_inputs, train_outputs, test_outputs = train_test_split(inputs, outputs, test_size=0.2, random_state=42)


In [4]:
from transformers import AutoTokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
import torch
from torch.utils.data import Dataset, DataLoader



class T5Dataset(Dataset):
    def __init__(self, inputs, outputs, tokenizer, max_length=512):
        self.inputs = inputs
        self.outputs = outputs
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, idx):
        input_text = self.inputs[idx]
        output_text = self.outputs[idx]

        # Tokenização
        input_encoding = self.tokenizer(
            input_text, padding='max_length', truncation=True, max_length=self.max_length, return_tensors='pt'
        )
        output_encoding = self.tokenizer(
            output_text, padding='max_length', truncation=True, max_length=self.max_length, return_tensors='pt'
        )

        input_ids = input_encoding['input_ids'].squeeze()
        attention_mask = input_encoding['attention_mask'].squeeze()
        labels = output_encoding['input_ids'].squeeze()

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'labels': labels
        }
    

model_name = "unicamp-dl/ptt5-base-portuguese-vocab"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name).to(device)
    
train_dataset = T5Dataset(train_inputs, train_outputs, tokenizer)
test_dataset = T5Dataset(test_inputs, test_outputs, tokenizer)

c:\Users\igorm\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\igorm\AppData\Local\Programs\Python\Python311\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained i

In [5]:
from transformers import AutoModelForSequenceClassification, Trainer, TrainingArguments
import os
import tempfile


temp_dir = tempfile.gettempdir()
results_dir = os.path.join(temp_dir, 'results')
logs_dir = os.path.join(temp_dir, 'logs')


def ensure_dir(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)


ensure_dir(results_dir)
ensure_dir(logs_dir)

print(f"Results directory exists: {os.path.exists(results_dir)}")
print(f"Logs directory exists: {os.path.exists(logs_dir)}")


training_args = TrainingArguments(
    output_dir=results_dir,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir=logs_dir,
    logging_steps=10,
    save_steps=10,
    evaluation_strategy="steps"
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
)


trainer.train()

model_save_path = "./results/t5_model"
tokenizer_save_path = "./results/t5_tokenizer"


os.makedirs(model_save_path, exist_ok=True)
os.makedirs(tokenizer_save_path, exist_ok=True)

model.save_pretrained(model_save_path)
tokenizer.save_pretrained(tokenizer_save_path)

print(f"Modelo salvo em {model_save_path}")
print(f"Tokenizador salvo em {tokenizer_save_path}")


Results directory exists: True
Logs directory exists: True


c:\Users\igorm\AppData\Local\Programs\Python\Python311\Lib\site-packages\transformers\training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
  3%|▎         | 10/369 [00:50<32:03,  5.36s/it]

{'loss': 36.4647, 'grad_norm': 101.56084442138672, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.08}


                                                
  3%|▎         | 10/369 [01:17<32:03,  5.36s/it]

{'eval_loss': 51.59592056274414, 'eval_runtime': 26.5647, 'eval_samples_per_second': 4.63, 'eval_steps_per_second': 1.167, 'epoch': 0.08}


...

 46%|████▌     | 170/369 [29:10<17:19,  5.22s/it]

{'loss': 0.8657, 'grad_norm': 1.4044634103775024, 'learning_rate': 1.7000000000000003e-05, 'epoch': 1.38}


                                                 
 46%|████▌     | 170/369 [29:23<17:19,  5.22s/it]

{'eval_loss': 0.5572430491447449, 'eval_runtime': 12.4913, 'eval_samples_per_second': 9.847, 'eval_steps_per_second': 2.482, 'epoch': 1.38}


SafetensorError: Error while serializing: IoError(Os { code: 112, kind: StorageFull, message: "There is not enough space on the disk." })

In [12]:
def generate_output(input_text):

    inputs = tokenizer.encode("translate English to English: " + input_text, return_tensors="pt", max_length=512, truncation=True)


    outputs = model.generate(inputs, max_length=512, num_beams=4, early_stopping=True)
    
    output_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    output_dict = {
        "catalog_catalog.code": "",
        "catalog_catalog.full_description": "",
        "catalog_catalog.description": "",
        "catalog_catalog.similar_parts": "N/D",
        "catalog_catalog.application_table": "",
        "catalog_catalog.brand_producer": "JEEP"
    }
    
    output_values = output_text.split(",")
    if len(output_values) == 5:
        output_dict["catalog_catalog.code"] = output_values[0].strip()
        output_dict["catalog_catalog.full_description"] = output_values[1].strip()
        output_dict["catalog_catalog.description"] = output_values[2].strip()
        output_dict["catalog_catalog.application_table"] = output_values[3].strip()
        output_dict["catalog_catalog.brand_producer"] = output_values[4].strip()

    return output_dict


input_text = "735598946\tCapa Espelho Retrovisor Direito Renegade 2019\tMarca: Jeep, Número de peça: 735598946, Modelo: RENEGADE  Anos: 2021, 2020, 2019, 2018, 2017, 2016, 2015..., Tipo de veículo: Carro/Caminhonete, Lado: Direito, Material: MTR, Origem: Brasil, \tDescriçãoSe aplica em:•JEEP Renegade de 2015 a 2021Codigos:735598946Peca Genuina, com nota fiscal e garantia!Em Caso de Duvidas, Consulte Nossos Especialistas:Para evitarmos erros e possiveis devolucoes, em caso de duvidas, informe sempre o Chassi Completo do seu veiculo no campo de perguntas. Assim poderemos verificar diretamente na fabrica a correta aplicacao para o seu veiculo.Seguranca:As Pecas Originais e Genuinas sao projetadas sob medida para cada modelo de automovel, elas sao a garantia de seguranca para voce. Todos nossos produtos sao originais e vao com nota fiscal.Suporte:- Temos Equipe de Atendimento e Suporte Tecnico Especializado a sua Disposicao.Saga, casa de amigos!Marca: Aplicacao universal ou nao informada, se tiver duvida, nos faca uma pergunta.________________Codigo:735598946________________ATENCAO:- Antes de efetuar a compra tire todas suas duvidas no campo perguntas.- Mantenha seus dados e enderecos atualizados no site do Mercado Livre.- Se o Mercado Envios nao entregar seu produto,seu pagamento sera devolvido automaticamente pelo Mercado Livre.- Temos carrinho de compras, adicione varios produtos e economize no Frete.- Todos os produtos anunciados estao disponiveis em nosso estoque.- Aceitamos devolucoes apenas com a embalagem Original e Nota Fiscal. - Devido a alteracao nas politicas do Mercado Livre o frete gratis esta sujeito ao peso, preco e distancia do envio.________________SUPORTE E HORARIOS DE ATENDIMENTO:- Temos Equipe de Atendimento e Suporte Tecnico Especializado a Disposicao.- Atendimento de Seg. a Sex. das 8h00 as 18h00 e Sab. 8h00 as 12h00.________________Garantia do vendedor: 90 dias"
output = generate_output(input_text)
print(output)

{'catalog_catalog.code': '', 'catalog_catalog.full_description': '', 'catalog_catalog.description': '', 'catalog_catalog.similar_parts': 'N/D', 'catalog_catalog.application_table': '', 'catalog_catalog.brand_producer': 'JEEP'}
